# **NYU Wagner - Python Coding for Public Policy**
# Class 5: APIs

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every pull; higher barrier to entry (reading documentation); subject to availability and performance of API

### Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

In [1]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population', match='Rank')
pop = tables[0]
pop

,Rank,Country or dependent territory,Region,Population,% of world,Date,Source (official or from the United Nations),Notes
0,–,NaN,World,7905246000,NaN,27 Oct 2021,UN projection[2],NaN
1,1,China,Asia,1411778724,NaN,1 Nov 2020,2020 census result[3],"The census figure refers to mainland China, ex..."
2,2,India,Asia,1383605415,NaN,27 Oct 2021,National population clock[4],The figure includes the population of India-ad...
3,3,United States,Americas,332604696,NaN,27 Oct 2021,National population clock[5],Includes the 50 states and the District of Col...
4,4,Indonesia,Asia,271350000,NaN,31 Dec 2020,National annual estimate[6],NaN
...,...,...,...,...,...,...,...,...
237,–,Niue (New Zealand),Oceania,1549,NaN,1 Jul 2021,National annual projection[91],NaN
238,–,Tokelau (New Zealand),Oceania,1501,NaN,1 Jul 2021,National annual projection[91],NaN
239,195,Vatican City,Europe,825,NaN,1 Feb 2019,Monthly national estimate[190],The total population of 825 consisted of 453 r...
240,–,Cocos (Keeling) Islands (Australia),Oceania,573,NaN,30 Jun 2020,National annual estimate[189],NaN


## Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Anchors aren't relevant

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the API key with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK)
   - Only if the API provider offers one
   - Abstracts the details away
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/)

In [2]:
import requests

params = {
    'api_key': 'DEMO_KEY',
    'q': 'Jimmy McMillan',
    'sort': '-first_file_date'
}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()
data

{'api_version': '1.0',
 'pagination': {'page': 1, 'per_page': 20, 'pages': 1, 'count': 2},
 'results': [{'first_file_date': '2015-10-13',
   'candidate_id': 'P60016805',
   'last_f2_date': '2015-10-13',
   'candidate_status': 'N',
   'incumbent_challenge': 'O',
   'load_date': '2018-02-17T09:16:20+00:00',
   'last_file_date': '2015-10-13',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'district_number': 0,
   'election_years': [2016],
   'district': '00',
   'office_full': 'President',
   'flags': 'P60016805',
   'state': 'US',
   'election_districts': ['00'],
   'incumbent_challenge_full': 'Open seat',
   'active_through': 2016,
   'has_raised_funds': False,
   'party': 'REP',
   'party_full': 'REPUBLICAN PARTY',
   'office': 'P',
   'candidate_inactive': False,
   'cycles': [2016, 2018],
   'inactive_election_years': None,
   'federal_funds_flag': False},
  {'first_file_date': '1995-03-08',
   'candidate_id': 'P60003290',
   'last_f2_date': '2011-02-07',
   'candidate_statu

### Retrieving nested data

In [3]:
data['results'][0]['name']

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### Reading into a DataFrame

In [4]:
params = {'api_key': 'DEMO_KEY'}
response = requests.get('https://api.open.fec.gov/v1/candidates/', params=params)
data = response.json()

pd.DataFrame(data['results'])

,candidate_status,incumbent_challenge_full,name,first_file_date,district_number,party,has_raised_funds,district,load_date,office_full,...,election_years,inactive_election_years,incumbent_challenge,candidate_id,active_through,last_file_date,candidate_inactive,flags,party_full,state
0,N,None,"12-INCH COCK PLEASE, CAN YOU SUCK MY",2021-04-28,0,PAF,False,00,2021-09-13T08:47:15+00:00,President,...,[2024],None,None,P40006033,2024,2021-05-05,False,P40006033,PEACE AND FREEDOM,US
1,N,Challenger,"753, JO",2019-04-23,0,NNE,False,00,2021-09-13T08:47:15+00:00,President,...,[2020],None,C,P00011569,2020,2019-04-23,False,P00011569,NONE,US
2,N,Challenger,"AABBATTE, MICHAEL THOMAS WITORT",2002-01-30,0,IND,False,00,2002-04-12T00:00:00+00:00,President,...,[2004],None,C,P40002172,2004,2002-01-30,False,P40002172,INDEPENDENT,US
3,C,Challenger,"AADLAND, ERIK",2021-06-04,0,REP,True,00,2021-09-13T08:47:16+00:00,Senate,...,[2022],None,C,S2CO00175,2022,2021-06-04,False,S2CO00175,REPUBLICAN PARTY,CO
4,C,Challenger,"AADLER, TIM",2020-03-24,3,REP,True,03,2020-05-05T21:11:57+00:00,House,...,[2020],None,C,H0UT03227,2020,2020-03-24,False,H0UT03227,REPUBLICAN PARTY,UT
5,N,Open seat,"AALDERS, TIM",None,4,IAP,False,04,2014-03-22T21:40:34+00:00,House,...,[2014],None,O,H4UT04052,2014,None,False,H4UT04052,INDEPENDENT AMERICAN PARTY,UT
6,P,Open seat,"AALDERS, TIMOTHY NOEL",2012-02-08,0,CON,True,00,2019-03-27T16:02:41+00:00,Senate,...,"[2012, 2018]",None,O,S2UT00229,2018,2018-04-23,False,S2UT00229,CONSTITUTION PARTY,UT
7,C,Open seat,"AALOORI, BANGAR REDDY",2019-10-17,22,REP,True,22,2020-03-18T21:13:37+00:00,House,...,[2020],None,O,H0TX22260,2020,2019-10-17,False,H0TX22260,REPUBLICAN PARTY,TX
8,P,None,"AAMODT, NORMAN O.",1976-04-12,16,REP,True,16,2002-03-30T00:00:00+00:00,House,...,"[1976, 1978]",None,None,H6PA16106,1978,1978-07-05,False,H6PA16106,REPUBLICAN PARTY,PA
9,P,Challenger,"AANESTAD, SAMUEL",2012-02-22,1,REP,True,01,2013-04-26T09:04:30+00:00,House,...,[2012],None,C,H2CA01110,2012,2012-02-22,False,H2CA01110,REPUBLICAN PARTY,CA


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2021, 10, 27, 18, 25, 47, 86002)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2021, 10, 20, 18, 25, 47, 86002)

In [7]:
start.isoformat()

'2021-10-20T18:25:47.086002'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = 'erm2-nwe9'
params = {
    '$where': f"created_date between '{start.isoformat()}' and '{now.isoformat()}'"
}

url = f'https://data.cityofnewyork.us/resource/{data_id}.json'
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '52243598',
  'created_date': '2021-10-20T18:25:58.000',
  'agency': 'DOHMH',
  'agency_name': 'Department of Health and Mental Hygiene',
  'complaint_type': 'Rodent',
  'descriptor': 'Rat Sighting',
  'location_type': '3+ Family Mixed Use Building',
  'incident_zip': '11385',
  'incident_address': '679 GRANDVIEW AVENUE',
  'street_name': 'GRANDVIEW AVENUE',
  'cross_street_1': 'GATES AVENUE',
  'cross_street_2': 'FOREST AVENUE',
  'intersection_street_1': 'GATES AVENUE',
  'intersection_street_2': 'FOREST AVENUE',
  'address_type': 'ADDRESS',
  'city': 'RIDGEWOOD',
  'landmark': 'GRANDVIEW AVENUE',
  'status': 'In Progress',
  'community_board': '05 QUEENS',
  'bbl': '4034880001',
  'borough': 'QUEENS',
  'x_coordinate_state_plane': '1010916',
  'y_coordinate_state_plane': '197033',
  'open_data_channel_type': 'PHONE',
  'park_facility_name': 'Unspecified',
  'park_borough': 'QUEENS',
  'latitude': '40.707446646356935',
  'longitude': '-73.90381958818121',
  'location'

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,closed_date,resolution_description,resolution_action_updated_date,facility_type,bridge_highway_name,road_ramp,bridge_highway_segment,taxi_pick_up_location,bridge_highway_direction,taxi_company_borough
0,52243598,2021-10-20T18:25:58.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Mixed Use Building,11385,679 GRANDVIEW AVENUE,GRANDVIEW AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52238182,2021-10-20T18:26:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,10026,302 WEST 112 STREET,WEST 112 STREET,...,2021-10-25T11:47:21.000,This service request was closed because the De...,2021-10-20T18:26:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52235873,2021-10-20T18:26:17.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10468,65 WEST KINGSBRIDGE ROAD,WEST KINGSBRIDGE ROAD,...,2021-10-20T18:39:58.000,The Police Department responded to the complai...,2021-10-20T18:40:02.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52240929,2021-10-20T18:26:23.000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,10467,3149 PERRY AVENUE,PERRY AVENUE,...,2021-10-20T19:03:29.000,The Police Department responded to the complai...,2021-10-20T19:03:33.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52239642,2021-10-20T18:26:27.000,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11101,PLACELLA PARK,PLACELLA PARK,...,2021-10-20T18:37:05.000,The Police Department responded to the complai...,2021-10-20T18:37:10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,52239248,2021-10-20T21:01:05.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11364,64-12B 223 PLACE,223 PLACE,...,2021-10-20T21:35:18.000,The Police Department responded and upon arriv...,2021-10-20T21:35:23.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,52236369,2021-10-20T21:01:07.000,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10308,329 TANGLEWOOD DRIVE,TANGLEWOOD DRIVE,...,2021-10-20T21:47:05.000,The Police Department responded and upon arriv...,2021-10-20T21:47:10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,52238767,2021-10-20T21:01:10.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10032,556 WEST 156 STREET,WEST 156 STREET,...,2021-10-20T21:42:33.000,The Police Department responded to the complai...,2021-10-20T21:42:38.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,52240889,2021-10-20T21:01:40.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11226,259 LINDEN BOULEVARD,LINDEN BOULEVARD,...,2021-10-20T21:24:08.000,The Police Department issued a summons in resp...,2021-10-20T21:24:12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

## Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## [Homework 5](https://padmgp-4506-spring.rcnyu.org/user-redirect/notebooks/class_materials/hw_5.ipynb)

## Homework 6

- a.k.a. the final project
- Open-ended
   - Be creative, and just the right amount of ambitious
- **Goal:** Prove or disprove a hypothesis using skills learned in this class

In real/ideal world, start with specific question and find data to answer it:

![project flow](https://textbook.coleridgeinitiative.org/ChapterIntro/figures/projectflow.png)

_Source: [Big Data and Social Science](https://textbook.coleridgeinitiative.org/chap-intro.html#the-structure-of-the-book)_

Data needed often doesn't exist or is hard (or impossible) to find/access

![project flow](https://textbook.coleridgeinitiative.org/ChapterIntro/figures/projectflow.png)

Safer approach, for the purposes of this assignment:

1. Find dataset that seems interesting
  - Use at least one dataset that you aren't familiar with, from this class or elsewhere.
  - [NYC OpenData](https://opendata.cityofnewyork.us/), [data.gov](https://www.data.gov/), and [Kaggle](https://www.kaggle.com/datasets) have many many options.
  - Finding a dataset available in CSV or JSON is recommended, though [pandas can read other formats](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).
1. Inspect the data a bit.
1. Come up with a question that the data is capable of answering and _isn't trivial to answer_.
  - If you aren't sure, ask.
1. Come up with a hypothesis (a.k.a. a guess of the answer to the question).
1. Submit the proposal.

### Proposal

Post responses to the following as a new Conversation under [the `HW6 proposals` Discussion](https://brightspace.nyu.edu/d2l/le/156784/discussions/topics/281277/View) by next class:

- **What dataset are you going to use?**
   - Please include a link. If multiple, how are you going to merge/join them?
- **What's the question you are trying to answer?**
   - It should be specific, and objectively answerable through the data available.
- **What's your hypothesis?**

- Your question/hypothesis doesn't need to be something novel; confirming something you read in the news is fine.
- You won't be graded on the scientific soundness of your work.
   - Important to think through and note assumptions/caveats of your approach
- The sooner you post your proposal, the sooner you'll get feedback.

#### Simplified example

- **Dataset:** [Recycling Diversion and Capture Rates](https://data.cityofnewyork.us/Environment/Recycling-Diversion-and-Capture-Rates/gaq9-z3hz)
- **Question:** From 2016 to 2019, what community district increased their diversion (recycling) rate the most?
- **Hypothesis:** [Bushwick](https://communityprofiles.planning.nyc.gov/brooklyn/4), because it's gentrified over that time, and hipsters love to recycle.

### Once you start

- Create a new notebook to do the actual analysis; you will turn that in separately.
- Go back and find any information that's available _around_ the data, to get a better understanding of what it contains and means.
  - Might include a data dictionary
  - Might involve poking around a government agency's web site to understand their processes
  - Understand what all the different columns and values represent

### Analysis requirements

_on top of [general assignment requirements](https://github.com/afeld/python-public-policy/blob/main/syllabus.md#assignments)_

<!-- make sure edits here are reflected in extras/scripts/hw_6_check.py -->

- **Read like a blog post**
    - Pretend you're explaining to a Wagner student who hasn't taken this class. You don't need to teach them Python, but they should be able to follow what's going on.
    - Re-state the question, hypothesis, and data source(s) with link(s)
    - Walk the reader through what you're doing in every step and what they should be taking away from it.
    - [Markdown](https://www.markdownguide.org/basic-syntax/) can be used in text cells for formatting.
    - Include any dead ends you hit.
    - Have a conclusion that speaks to your question and hypothesis.
- **Use pandas**
- **Not be trivial**, requiring:
    - At least 40 lines of code to come to a conclusion
    - Transforming data through [grouping](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html), [merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging), and/or [reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html) of DataFrames
    - Operations that aren't easily done in a spreadsheet.
- **Have a visualization** (chart or map) of some kind
- Don't leave any sensitive information in the notebook: API keys, personally-identifiable information (PII), etc.

If you answer the first question easily, that's fine; dig into / build off of it. Go deep, not broad.

## No homework/resubmissions will be accepted after Thursday 5/12 at 6:45pm ET

In other words, homework 6 cannot be late.

## Lecture 6